### Importing python modules

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time 
%matplotlib inline
import sys 
import re
from random import randint
import random 
from time import sleep

import dateutil.parser

from bs4 import BeautifulSoup


#### use http://www.boxofficemojo.com/robots.txt to see access to which sites are blocked and for other info

### Functions to parse data

In [128]:
def to_date(datestring):
    try:
        date = dateutil.parser.parse(datestring)
        return date
    except:
        return None
    

def money_to_int(moneystring):
    try:
        moneystring = moneystring.replace('$', '').replace(',', '')
        return int(moneystring)
    except:
        try:
            if 'million' in moneystring or 'mil' in moneystring:
                moneystring = moneystring.replace('$', '').replace(',', '').replace('million','').replace('mil','').replace('mill','')
                moneyInMil = int(moneystring)*1000000
                return moneyInMil
        except:
            return None
    
def runtime_to_minutes(runtimestring):
    runtime = runtimestring.split()
    try:
        minutes = int(runtime[0])*60 + int(runtime[2])
        return minutes
    except:
        return None

#### Go to main starting page from where, click on each year and get information about movies for that particular year


In [5]:
# if needed: pip install requests
import requests

mainURL = 'http://www.boxofficemojo.com/yearly/'

headers = {
    'user-Agent': 'Mozilla/5.0'
    
}

response = requests.get(mainURL, headers = headers)



In [6]:
response.status_code

200

In [7]:
page = response.text
soup = BeautifulSoup(page)
#print(soup.prettify())

/Users/deepakgautam/anaconda/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


#### Get basic info for each year along with links to enther individual year page

In [81]:
page_1_TrList = soup.find_all('table')[1].find_all('table')[0].find_all('tr')

overallTable_df = pd.DataFrame(columns=('Year', 'Gross', 'GrossChange','TicketSold','TSChange','TotalMovies','TotalScreens','AvgTicketPrice','AvgCost','TopMovie'))
page1_LinkList =[]
#skipping first row as it contains header items
for rowCount, tr in enumerate(page_1_TrList[1:]):
    tdList = tr.find_all('td')
    colVals = []
    
    for tdIndex, td in enumerate(tdList):

        #getting redirect link from the first item
        if tdIndex ==0:
            smLink = td.find_all('font')[0].find('a')['href']
            fullLink = mainURL + smLink
        
        item = td.find_all('font')[0].text
        
        colVals.append(item)   
    
    page1_LinkList.append(fullLink)
    overallTable_df.loc[rowCount] = colVals

    

In [88]:
# Write Overall Yearly Table to CSV file
overallTable_df.to_csv('/Users/deepakgautam/BasicMovieAnalytics/Files/yearSummary.csv')

In [104]:
errorPageList = []

#### Function which takes a link for yearly page and find individual movies info for that year and also links for those movies


In [105]:
def getDataFromByYearPages(inputURL ,releaseYear):
    #movie row starts from index 2 and goes up to index 101
    
    headers = {
        'user-Agent': 'Mozilla/5.0'
    }
    
    #print(inputURL)
    response = requests.get(inputURL, headers = headers)
    
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    
    columns=('Rank', 'MovieName', 'Studio','TotalDomestic','TheaterCountTotal','TotalOpeningWeekend','TheaterCountOW','ReleaseDate')
    
    temp_allMovieURLList = []
    

    #movie row starts from index 2 and goes up to index 
    
    try:
        totalRows = len(soup.find_all('table')[4].find_all('table')[1].find_all('tr'))
    except:
        totalRows = 0
        errorPageList.append(inputURL)
    
    data = []
    for i in range(2,totalRows-4):
        
        curTDList = soup.find_all('table')[4].find_all('table')[1].find_all('tr')[i].find_all('td')
        colVals = []
        for tdIndex, td in enumerate(curTDList):
            
            if tdIndex > 7:
                break # some pages have closing dates and some page do not .. so we will stop after 
            
            #getting redirect link from the first item
            if tdIndex ==1:
                
                try:
                    smLink = td.find_all('font')[0].find('a')['href']
                    movieURL = 'http://www.boxofficemojo.com' + str(smLink)

                    if movieURL not in allMovieURLList and movieURL not in temp_allMovieURLList:
                        temp_allMovieURLList.append(movieURL)
                except:
                     e = sys.exc_info()[0]
                    
            
            item = td.find_all('font')[0].text
            
            if tdIndex in range(3,7):
                item = money_to_int(item)
            
            if tdIndex ==7:
                item = item + '/' + str(releaseYear) # since opening week date does not have year info
                item = to_date(item)
                
            colVals.append(item)
        
        #print(i)
        data.append(colVals)
        
    return(pd.DataFrame(data, columns=columns), temp_allMovieURLList)

    

### loop through each link list(yearly) and get lists of movies for that year



In [106]:
# instantiate a list which keeps track of URL for every movie for every year that we are looping through

allMovieURLList = []
main_movie_df = pd.DataFrame(columns=('Rank', 'MovieName', 'Studio','TotalDomestic','TheaterCountTotal','TotalOpeningWeekend','TheaterCountOW','ReleaseDate'))

successFulLinks_page1 = []
data_dfs = []
temp_links = []

for pageLink in page1_LinkList: # remove :1 when you have dealt response of one page
    time.sleep(random.uniform(1,3))
    mainURL = pageLink

    headers = {
        'user-Agent': 'Mozilla/5.0'
    }
    response = requests.get(mainURL, headers = headers)
    
    #print(pageLink)
    if response.status_code == 200:
        successFulLinks_page1.append(True)
        
        page = response.text
        soup = BeautifulSoup(page, "lxml")
        
        
        # soup.find_all('table')[3] has information about different pages in each year.
        rawPageList = soup.find_all('table')[3].find_all('tr')[0].find('td').find('b').find_all('a')
        pageList_inPage2 = []
        
        for curList in rawPageList:
            pageList_inPage2.append(str('http://www.boxofficemojo.com/' + curList['href']))
            
        
        
        # call a function that gets page2 _ current page_ movie link and gives back list of movies and info in a dataset
        # Remember that you will update allMovieURLList from this function
        # Append this to main Movie Table
        
        temp_allMovieURLList = []
        
        curYear = mainURL[-11:-7]
        tempMovie_df, temp_allMovieURLList = getDataFromByYearPages(mainURL, curYear)
        
        data_dfs.append(tempMovie_df)        
        temp_links = temp_links + temp_allMovieURLList
        
        # Then loop thorugh all items pageList_inPage2 and call the same function as above and save their info in Movie table
        temp_allMovieURLList = []
        for curPage in pageList_inPage2:
            time.sleep(1)
            tempMovie_df, temp_allMovieURLList = getDataFromByYearPages(curPage,curYear)
            
            data_dfs.append(tempMovie_df)
            temp_links = temp_links + temp_allMovieURLList
    else:
        successFulLinks_page1.append(False)

main_movie_df = pd.concat(data_dfs)

allMovieURLList = temp_links

http://www.boxofficemojo.com/yearly/chart/?yr=2016&p=.htm
http://www.boxofficemojo.com/yearly/chart/?yr=2015&p=.htm
http://www.boxofficemojo.com/yearly/chart/?yr=2014&p=.htm
http://www.boxofficemojo.com/yearly/chart/?yr=2013&p=.htm
http://www.boxofficemojo.com/yearly/chart/?yr=2012&p=.htm
http://www.boxofficemojo.com/yearly/chart/?yr=2011&p=.htm
http://www.boxofficemojo.com/yearly/chart/?yr=2010&p=.htm
http://www.boxofficemojo.com/yearly/chart/?yr=2009&p=.htm
http://www.boxofficemojo.com/yearly/chart/?yr=2008&p=.htm
http://www.boxofficemojo.com/yearly/chart/?yr=2007&p=.htm
http://www.boxofficemojo.com/yearly/chart/?yr=2006&p=.htm
http://www.boxofficemojo.com/yearly/chart/?yr=2005&p=.htm
http://www.boxofficemojo.com/yearly/chart/?yr=2004&p=.htm
http://www.boxofficemojo.com/yearly/chart/?yr=2003&p=.htm
http://www.boxofficemojo.com/yearly/chart/?yr=2002&p=.htm
http://www.boxofficemojo.com/yearly/chart/?yr=2001&p=.htm
http://www.boxofficemojo.com/yearly/chart/?yr=2000&p=.htm
http://www.box

In [110]:
## write mainmovie table and all movie url into csv file 
#main_movie_df.to_csv('/Users/deepakgautam/sf16_ds3/projects/02-luther/local/allMovies.csv')


14095

### Function to get additional info for individual movie from the individual movie link

In [112]:
def getMovieValues(soup):
    valList = []
    
    columns=('Movie', 'Distributor','Genre','RunTime','MPAA','Budget','ForeignSale')
    
    try:
        curMovieName = soup.find_all('font')[1].text
    except:
        curMovieName = None

    try:
        #soup.find_all('table')[5].find_all('tr')[0].find('b').text
        curDist = soup.find_all('table')[5].find_all('tr')[1].find_all('b')[0].text
    except:
        curDist= None

    try:
        curGenre = soup.find_all('table')[5].find_all('tr')[2].find_all('b')[0].text
    except:
        curGenre= None

    try:
        raw_curRuntime = soup.find_all('table')[5].find_all('tr')[2].find_all('b')[1].text
        curRuntime = runtime_to_minutes(raw_curRuntime)
    except:
        curRuntime= None

    try:
        curMPAA = soup.find_all('table')[5].find_all('tr')[3].find_all('b')[0].text
    except:
        curMPAA = None

    try:
        raw_curBudget = soup.find_all('table')[5].find_all('tr')[3].find_all('b')[1].text
        curBudget = money_to_int(raw_curBudget)
    except:
        curBudget= None

    try:
        raw_ForeignSale = soup.find_all('table')[9].find_all('tr')[1].find_all('td')[1].text
        ForeignSale = money_to_int(raw_ForeignSale)
    except:
        ForeignSale = None
       
    valList.append(curMovieName)
    valList.append(curDist)
    valList.append(curGenre)
    valList.append(curRuntime)
    valList.append(curMPAA)
    valList.append(curBudget)
    valList.append(ForeignSale)
    
    
    return(pd.DataFrame([valList], columns=columns))
    
    

### Loop through values of allMovieURLList to get individual movie's additional info

In [156]:

Ind_movie_df = pd.DataFrame(columns=('Movie', 'Distributor','Genre','RunTime','MPAA','Budget','ForeignSale'))

successFulLinks_IndPages = []
data_dfs = []
count = 0
errorPageList_indMovies = []

for pageLink in allMovieURLList: # remove :1 when you have dealt response of one page
    time.sleep(random.uniform(0.5,1.5))
    mainURL = pageLink

    headers = {
        'user-Agent': 'Mozilla/5.0'
    }
    response = requests.get(mainURL, headers = headers)
    
    #print(pageLink)
    if response.status_code == 200:
        successFulLinks_IndPages.append(True)
        
        page = response.text
        soup = BeautifulSoup(page, "lxml")
        
        try:
            tempMovie_df = getMovieValues(soup)
        except:
            errorPageList_indMovies.append(pageLink)
        
        data_dfs.append(tempMovie_df)
        
    else:
        successFulLinks_IndPages.append(False)
        
    # in every 500 movies write to csv file, so that if you lose connection you at least have a file
    
    
    if count > 1 and count%500 == 0:
        Ind_movie_df = pd.concat(data_dfs)
        Ind_movie_df.to_csv('/Users/deepakgautam/BasicMovieAnalytics/Files/indMovieInfo.csv')
        #print(count)
        
    if count > 1  and count%10 == 0:
        with open("/Users/deepakgautam/BasicMovieAnalytics/Files/movieProgress.txt", "a") as myfile:
            myfile.write("Scrapping complete for " + str(count + 1) + " movies. \n")
    
    count += 1 
    
        
Ind_movie_df = pd.concat(data_dfs)





12500
13000
13500
14000


In [158]:
## To save Individual Movie info as csv file
#Ind_movie_df.to_csv('/Users/deepakgautam/BasicMovieAnalytics/Files/indMovieInfo.csv')


In [165]:
### To save all movie url List
## pd.DataFrame({"movie_name": allMovieURLList}).to_csv("/Users/deepakgautam/BasicMovieAnalytics/Files/allMovieURL_List.csv")